### Config file used for training

---

Segmentation: binary

Model:

      name: ReFineNetLite
      param:
Criterion:

    name: BinaryCrossEntropy
    param:
Loader:

    root: path/to/data/folder
    image_normalizer: divide_by_255
    label_normalizer: binary_label
    batch_size: 2

Metrics:

      - accuracy
      - precision
      - f1
      - recall
      - iou
Optimizer:

      name: Adam
      param:
Callbacks:
      
      # Ouptut from all the callbacks caller will be stored at the path specified in log_dir
      log_dir: path/where/to/log
      callers:
        - TrainChkCallback
        - TimeCallback
        - TensorBoardCallback
        - TrainStateCallback


In [1]:
import yaml
import os
import sys
import albumentations as A

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from building_footprint_segmentation.segmentation import init_segmentation, read_trainer_config
from building_footprint_segmentation.helpers.callbacks import CallbackList, load_callback
from building_footprint_segmentation.trainer import Trainer

config_path = r"D:\Cypherics\Library\building-footprint-segmentation\config.yaml"

### Load the config file for training

In [2]:
config = read_trainer_config(config_path)

### Initalize type of segmentation to run for building footprint

In [3]:
segmentation = init_segmentation(config["Segmentation"])

### Load essential segmentation objects for training

In [4]:
augmenters = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2)
])

model = segmentation.load_model(name=config["Model"]["name"])
criterion = segmentation.load_criterion(name=config["Criterion"]["name"])
loader = segmentation.load_loader(
    config["Loader"]["root"],
    config["Loader"]["image_normalizer"],
    config["Loader"]["label_normalizer"],
    augmenters,
    config["Loader"]["batch_size"],
)
metrics = segmentation.load_metrics(
    data_metrics=config["Metrics"]
)

optimizer = segmentation.load_optimizer(model, name=config["Optimizer"]["name"])

callbacks = CallbackList()
# Ouptut from all the callbacks caller will be stored at the path specified in log_dir
for caller in config["Callbacks"]["callers"]:
    callbacks.append(load_callback(config["Callbacks"]["log_dir"], caller))

### Initalize the Trainer object


In [5]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    loader=loader,
    metrics=metrics,
    callbacks=callbacks,
    optimizer=optimizer,
    scheduler=None,
)

### Start training


In [6]:
trainer.train(start_epoch=0, end_epoch=2)

Epoch 0, lr 0.001:   0%|                                                                                                                                                                      | 0/8 [00:00<?, ?it/s]C:\Users\Fuzail\anaconda3\envs\building-footprint-segmentation\lib\site-packages\torch\nn\functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Epoch 0, lr 0.001: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:12<00:00,  9.05s/it, loss=0.64960]

>>Py-OneLiner : Validation : 1/1 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 0.6495960652828217, validation: 0.36756959557533264
>>Py-OneLiner : Train Metric : accuracy : 0.79162, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.96608, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000

Epoch 1, lr 0.001: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:13<00:00,  9.22s/it, loss=0.55847]

>>Py-OneLiner : Validation : 1/1 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 0.5584663450717926, validation: 0.1944824457168579
>>Py-OneLiner : Train Metric : accuracy : 0.79162, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.96608, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 0:Hours 3:Minutes 8:Seconds

### Resuming Training


In [7]:
path_to_saved_state = r"D:\Cypherics\lib_check\out_data\state\default.pt"

# if new_end_epoch is None, the trainer will resume and run till end_epoch, the value of which is saved in state file 
trainer.resume(state=path_to_saved_state, new_end_epoch=4)

Epoch 2, lr 0.001:   0%|                                                                                                                                                                      | 0/8 [00:00<?, ?it/s]C:\Users\Fuzail\anaconda3\envs\building-footprint-segmentation\lib\site-packages\torch\nn\functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Epoch 2, lr 0.001: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:58<00:00,  7.31s/it, loss=0.49544]

>>Py-OneLiner : Validation : 1/1 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 0.49543876200914383, validation: 0.24815678596496582
>>Py-OneLiner : Train Metric : accuracy : 0.79162, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.96608, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000

Epoch 3, lr 0.001: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:59<00:00,  7.45s/it, loss=0.56119]

>>Py-OneLiner : Validation : 1/1 ETA -- 0:Days 0:Hours 0:Minutes 0:Seconds

>>Py-OneLiner : Loss : train: 0.5611927509307861, validation: 0.3014044165611267
>>Py-OneLiner : Train Metric : accuracy : 0.79162, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Valid Metric : accuracy : 0.96608, precision : 0.00000, f1 : 0.00000, recall : 0.00000, iou : 0.00000
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 0:Hours 2:Minutes 37:Seconds